In [21]:
import pandas as pd
from scipy.stats import mode
import numpy as np
import itertools

In [22]:
files = [
    # "more_universities_bs_0925.xlsx",
    # "more_universities_bs_0925_2.xlsx",
    # "more_universities_bs_0925_3.xlsx",
    # "more_universities_second_half_bs_0929.xlsx",
    # "more_universities_second_half_bs_0929_2.xlsx",
    # "more_universities_second_half_bs_0929_3.xlsx",
    # "more_universities_second_half_bs_0929_4.xlsx"
    # "more_universities_second_half_bs_1009.xlsx",
    # "more_universities_second_half_bs_1009_2.xlsx",
    # "more_universities_second_half_bs_1009_3.xlsx"
    "more_universities_second_half_bs_1013.xlsx",
    "more_universities_second_half_bs_1013_2.xlsx",
    "more_universities_second_half_bs_1013_3.xlsx"
]

dfs = [pd.read_excel(f, index_col=0) for f in files]
combined = pd.concat(dfs, keys=range(len(dfs)))


def safe_mode(x):
    counts = x.value_counts(dropna=True)
    if counts.empty:
        return 0

    # Case 1: No unique mode (tie in counts)
    if len(counts) > 1 and counts.iloc[0] == counts.iloc[1:].max():
        unique_vals = np.sort(x.dropna().unique())
        
        # --- Rule A: All 3 values within 1% of their average ---
        if len(unique_vals) == 3:
            avg_all = np.mean(unique_vals)
            diffs = np.abs(unique_vals - avg_all) / avg_all
            if (diffs < 0.01).all():
                return avg_all

            # --- Rule B: Any 2 of 3 values within 5% of each other ---
            for a, b in itertools.combinations(unique_vals, 2):  # all (n choose 2) pairs
                if abs(a - b) / np.mean([a, b]) < 0.05:
                    return np.mean([a, b])
                
        # --- Rule C: if no mode is found, return the latest one
        return x.iloc[-1]

    return counts.idxmax()

In [23]:
combined = combined.copy()
combined["RowType"] = "data"
combined = combined.reset_index(level=0, drop=True)

In [24]:
if isinstance(combined.columns, pd.MultiIndex):
    combined.columns = combined.columns.droplevel(0)
if 'school' in combined.index.names:
    combined = combined.reset_index(level='school')

In [25]:
# combined['receivables_leftover'].fillna(combined['receivables_leftover_calculated'])

net_receivables_components = [
        "accounts_receivable",
        "pledges_receivable",
        "government_grants_and_other_receivables",
        "loans_receivable",
        # "receivables_leftover"
        "receivables_leftover_calculated"
    ]

combined['net_receivables'] = combined[net_receivables_components].fillna(0).sum(axis=1)

In [26]:
cols = [
    "accumulated_depreciation_bs",
    "accumulated_depreciation_notes",
    "accumulated_amortization_bs",
    "accumulated_amortization_notes"
]
for col in cols:
    combined[col] = pd.to_numeric(combined[col], errors="coerce").fillna(0)

combined["accumulated_depreciation"] = (
    combined[["accumulated_depreciation_bs", "accumulated_depreciation_notes"]]
    .replace(0, np.nan)
    .bfill(axis=1)
    .iloc[:, 0]
    .fillna(0)
    +
    combined[["accumulated_amortization_bs", "accumulated_amortization_notes"]]
    .replace(0, np.nan)
    .bfill(axis=1)
    .iloc[:, 0]
    .fillna(0)
)

In [27]:
mode_df = combined.groupby(['school'], dropna=False).agg(safe_mode).reset_index()
mode_df["RowType"] = "mode"

In [28]:
final_df = pd.concat([combined, mode_df], axis=0)

In [29]:
# final_df["_sort_key"] = final_df.index.astype(str) + final_df["RowType"].map(lambda x: "zzz" if x=="mode" else "aaa")
# final_df = final_df.sort_values(by=["_sort_key"], kind="mergesort").drop(columns="_sort_key")

final_df["_sort_key"] = final_df["RowType"].map(lambda x: 1 if x == "data" else 2)
final_df = (
    final_df.sort_values(by=["school", "_sort_key"], kind="mergesort")
             .drop(columns="_sort_key")
             .reset_index(drop=True)
)

In [30]:
asset_components = [
    "cash_and_short_term_investments_unrestricted",
    "net_receivables",
    "net_fixed_assets",
    "long_term_investments_unrestricted_and_restricted",
    "rou_assets_operating_lease"
]

liability_components = [
    "short_term_debt",
    # "current_portion_finance_lease",
    # "current_portion_long_term_debt",
    # "current_portion_operating_lease",
    "accounts_payable",
    "deferred_revenue",
    "long_term_debt",
    "finance_lease_liability",
    "operating_lease_liability",
    "swap_obligation_fmv",
    "pension_and_opeb_liability"
]

final_df["sum_asset_components"] = final_df[asset_components].fillna(0).sum(axis=1)
final_df["sum_liability_components"] = final_df[liability_components].fillna(0).sum(axis=1)

mask_mode = final_df["RowType"] == "mode"

final_df.loc[mask_mode, "other_assets_plug"] = (
    final_df.loc[mask_mode, "total_assets"] 
    - final_df.loc[mask_mode, "sum_asset_components"]
)

final_df.loc[mask_mode, "other_liabilities_plug"] = (
    final_df.loc[mask_mode, "total_liabilities"] 
    - final_df.loc[mask_mode, "sum_liability_components"]
)

final_df.drop(columns=["sum_asset_components", "sum_liability_components"], inplace=True)

In [31]:
order = ['RowType']+[col for col in final_df.columns if col != 'RowType']
final_df_ordered = final_df[order]
final_df_ordered.to_excel("combined_with_modes_1012.xlsx",index=False)